# 06 CNN 反向传播

> 推导卷积层和池化层的梯度

---

## 📚 学习目标

- [ ] 推导卷积层对权重的梯度
- [ ] 推导卷积层对输入的梯度
- [ ] 理解池化层的梯度传播
- [ ] 实现 CNN 反向传播

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(42)

---

## 📐 卷积层的梯度推导

### 前向传播回顾

$$Y[i, j] = \sum_{m} \sum_{n} X[i+m, j+n] \cdot W[m, n] + b$$

### 问题

已知上游梯度 $\frac{\partial L}{\partial Y}$，求：
1. $\frac{\partial L}{\partial W}$（权重梯度，用于更新）
2. $\frac{\partial L}{\partial X}$（输入梯度，用于传递）
3. $\frac{\partial L}{\partial b}$（偏置梯度）

### 推导1：对权重 W 的梯度

由链式法则：

$$\frac{\partial L}{\partial W[m, n]} = \sum_{i} \sum_{j} \frac{\partial L}{\partial Y[i,j]} \cdot \frac{\partial Y[i,j]}{\partial W[m,n]}$$

因为 $\frac{\partial Y[i,j]}{\partial W[m,n]} = X[i+m, j+n]$，所以：

$$\frac{\partial L}{\partial W[m, n]} = \sum_{i} \sum_{j} \frac{\partial L}{\partial Y[i,j]} \cdot X[i+m, j+n]$$

**这就是 X 和 dY 的卷积！**

$$\frac{\partial L}{\partial W} = X * \frac{\partial L}{\partial Y}$$

### 推导2：对输入 X 的梯度

$$\frac{\partial L}{\partial X[i, j]} = \sum \frac{\partial L}{\partial Y} \cdot \frac{\partial Y}{\partial X[i,j]}$$

经过推导（较复杂）：

$$\frac{\partial L}{\partial X} = \frac{\partial L}{\partial Y} *_{full} W^{rot180}$$

其中：
- $*_{full}$ 表示"full" 模式卷积
- $W^{rot180}$ 表示核旋转 180°

In [ ]:
# ============================================================
# 卷积层反向传播实现
# ============================================================

def conv_backward(dY, X, W):
    """
    卷积层的反向传播
    
    参数:
        dY: 上游梯度, shape (out_h, out_w)
        X: 前向传播时的输入, shape (H, W)
        W: 权重, shape (k_h, k_w)
    
    返回:
        dW: 权重梯度, shape (k_h, k_w)
        dX: 输入梯度, shape (H, W)
        db: 偏置梯度, 标量
    """
    H, W_in = X.shape
    k_h, k_w = W.shape
    out_h, out_w = dY.shape
    
    # ========================================
    # 偏置梯度：dY 的所有元素之和
    # ========================================
    db = np.sum(dY)
    
    # ========================================
    # 权重梯度：X 和 dY 的卷积
    # dW[m,n] = Σ_i Σ_j dY[i,j] · X[i+m, j+n]
    # ========================================
    dW = np.zeros_like(W)
    for m in range(k_h):
        for n in range(k_w):
            # 提取 X 中对应的区域
            X_region = X[m:m+out_h, n:n+out_w]
            dW[m, n] = np.sum(dY * X_region)
    
    # ========================================
    # 输入梯度：dY 与旋转后的 W 的 full 卷积
    # ========================================
    # 旋转 W 180°
    W_rot = np.rot90(W, 2)
    
    # 对 dY 进行 padding
    pad = k_h - 1
    dY_padded = np.pad(dY, pad, mode='constant', constant_values=0)
    
    dX = np.zeros((H, W_in))
    for i in range(H):
        for j in range(W_in):
            window = dY_padded[i:i+k_h, j:j+k_w]
            dX[i, j] = np.sum(window * W_rot)
    
    return dW, dX, db

In [ ]:
# ============================================================
# 梯度检验：验证反向传播正确性
# ============================================================

def numerical_gradient(f, x, eps=1e-5):
    """数值梯度（中心差分）"""
    grad = np.zeros_like(x)
    it = np.nditer(x, flags=['multi_index'])
    
    while not it.finished:
        idx = it.multi_index
        old_val = x[idx]
        
        x[idx] = old_val + eps
        fxh1 = f(x)
        
        x[idx] = old_val - eps
        fxh2 = f(x)
        
        grad[idx] = (fxh1 - fxh2) / (2 * eps)
        x[idx] = old_val
        
        it.iternext()
    
    return grad

# 测试
X = np.random.randn(5, 5)
W = np.random.randn(3, 3)
b = 0.0

def forward(X):
    out_h, out_w = 3, 3
    Y = np.zeros((out_h, out_w))
    for i in range(out_h):
        for j in range(out_w):
            Y[i, j] = np.sum(X[i:i+3, j:j+3] * W) + b
    return Y

Y = forward(X)
dY = np.ones_like(Y)  # 假设损失对 Y 的梯度全为 1

# 解析梯度
dW_analytical, dX_analytical, db_analytical = conv_backward(dY, X, W)

# 数值梯度（对 W）
def loss_w(W_flat):
    W_temp = W_flat.reshape(3, 3)
    out_h, out_w = 3, 3
    Y = np.zeros((out_h, out_w))
    for i in range(out_h):
        for j in range(out_w):
            Y[i, j] = np.sum(X[i:i+3, j:j+3] * W_temp) + b
    return np.sum(Y)

dW_numerical = numerical_gradient(loss_w, W.flatten().copy()).reshape(3, 3)

print("梯度检验 (对 W)")
print(f"解析梯度:\n{dW_analytical}")
print(f"\n数值梯度:\n{dW_numerical}")
print(f"\n相对误差: {np.max(np.abs(dW_analytical - dW_numerical)) / np.max(np.abs(dW_numerical)):.2e}")

---

## 📐 池化层的梯度

### 最大池化的梯度

梯度只传递给产生最大值的位置，其他位置梯度为 0。

```
前向:           反向:
[1, 3]          [0, dY]
[2, 4] → 4      [0, 0 ]
```

In [ ]:
def maxpool_backward(dY, X, pool_size=2):
    """
    最大池化的反向传播
    
    参数:
        dY: 上游梯度, shape (out_h, out_w)
        X: 前向传播时的输入, shape (H, W)
        pool_size: 池化窗口大小
    
    返回:
        dX: 输入梯度, shape (H, W)
    """
    H, W = X.shape
    out_h, out_w = dY.shape
    
    dX = np.zeros_like(X)
    
    for i in range(out_h):
        for j in range(out_w):
            # 获取池化窗口
            i_start = i * pool_size
            j_start = j * pool_size
            window = X[i_start:i_start+pool_size, j_start:j_start+pool_size]
            
            # 找到最大值的位置
            max_idx = np.unravel_index(np.argmax(window), window.shape)
            
            # 梯度只传递给最大值位置
            dX[i_start + max_idx[0], j_start + max_idx[1]] = dY[i, j]
    
    return dX

# 测试
X_pool = np.array([
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
    [13, 14, 15, 16]
], dtype=float)

dY_pool = np.array([
    [1, 1],
    [1, 1]
], dtype=float)

dX_pool = maxpool_backward(dY_pool, X_pool)

print("输入 X:")
print(X_pool)
print("\n上游梯度 dY:")
print(dY_pool)
print("\n输入梯度 dX (只有最大值位置有梯度):")
print(dX_pool)

---

## 📝 本章小结

### 核心公式

| 层 | 公式 |
|------|------|
| Conv 权重梯度 | $dW = X * dY$ |
| Conv 输入梯度 | $dX = dY *_{full} W^{rot180}$ |
| Conv 偏置梯度 | $db = \sum dY$ |
| MaxPool 梯度 | 只传给最大值位置 |
| AvgPool 梯度 | 均匀分配到所有位置 |

### 关键洞察

**卷积的反向传播也是卷积！** 这就是为什么 CNN 的训练可以高效实现。

---

## 🚀 下一步

继续学习 **07_classic_architectures.ipynb** - 经典 CNN 架构